Deep Learning
=============

Assignment 4
------------

Previously in `2_fullyconnected.ipynb` and `3_regularization.ipynb`, we trained fully connected networks to classify [notMNIST](http://yaroslavvb.blogspot.com/2011/09/notmnist-dataset.html) characters.

The goal of this assignment is make the neural network convolutional.

In [16]:
# These are all the modules we'll be using later. Make sure you can import them
# before proceeding further.
from __future__ import print_function
import numpy as np
import tensorflow as tf
from six.moves import cPickle as pickle
from six.moves import range

In [17]:
pickle_file = 'notMNIST.pickle'

with open(pickle_file, 'rb') as f:
  save = pickle.load(f)
  train_dataset = save['train_dataset']
  train_labels = save['train_labels']
  valid_dataset = save['valid_dataset']
  valid_labels = save['valid_labels']
  test_dataset = save['test_dataset']
  test_labels = save['test_labels']
  del save  # hint to help gc free up memory
  print('Training set', train_dataset.shape, train_labels.shape)
  print('Validation set', valid_dataset.shape, valid_labels.shape)
  print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 28, 28) (200000,)
Validation set (10000, 28, 28) (10000,)
Test set (10000, 28, 28) (10000,)


Reformat into a TensorFlow-friendly shape:
- convolutions need the image data formatted as a cube (width by height by #channels)
- labels as float 1-hot encodings.

In [18]:
image_size = 28
num_labels = 10
num_channels = 1 # grayscale

import numpy as np

def reformat(dataset, labels):
  dataset = dataset.reshape(
    (-1, image_size, image_size, num_channels)).astype(np.float32)
  labels = (np.arange(num_labels) == labels[:,None]).astype(np.float32)
  return dataset, labels
train_dataset, train_labels = reformat(train_dataset, train_labels)
valid_dataset, valid_labels = reformat(valid_dataset, valid_labels)
test_dataset, test_labels = reformat(test_dataset, test_labels)
print('Training set', train_dataset.shape, train_labels.shape)
print('Validation set', valid_dataset.shape, valid_labels.shape)
print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 28, 28, 1) (200000, 10)
Validation set (10000, 28, 28, 1) (10000, 10)
Test set (10000, 28, 28, 1) (10000, 10)


In [19]:
def accuracy(predictions, labels):
  return (100.0 * np.sum(np.argmax(predictions, 1) == np.argmax(labels, 1))
          / predictions.shape[0])

Let's build a small network with two convolutional layers, followed by one fully connected layer. Convolutional networks are more expensive computationally, so we'll limit its depth and number of fully connected nodes.

In [7]:
batch_size = 16
patch_size = 5
depth = 16
num_hidden = 64

graph = tf.Graph()

with graph.as_default():

  # Input data.
  tf_train_dataset = tf.placeholder(
    tf.float32, shape=(batch_size, image_size, image_size, num_channels))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  # Variables.
  layer1_weights = tf.Variable(tf.truncated_normal(
      [patch_size, patch_size, num_channels, depth], stddev=0.1))
  layer1_biases = tf.Variable(tf.zeros([depth]))
  layer2_weights = tf.Variable(tf.truncated_normal(
      [patch_size, patch_size, depth, depth], stddev=0.1))
  layer2_biases = tf.Variable(tf.constant(1.0, shape=[depth]))
  layer3_weights = tf.Variable(tf.truncated_normal(
      [image_size // 4 * image_size // 4 * depth, num_hidden], stddev=0.1))
  layer3_biases = tf.Variable(tf.constant(1.0, shape=[num_hidden]))
  layer4_weights = tf.Variable(tf.truncated_normal(
      [num_hidden, num_labels], stddev=0.1))
  layer4_biases = tf.Variable(tf.constant(1.0, shape=[num_labels]))
  
  # Model.
  def model(data):
    conv = tf.nn.conv2d(data, layer1_weights, [1, 2, 2, 1], padding='SAME')
    hidden = tf.nn.relu(conv + layer1_biases)
    conv = tf.nn.conv2d(hidden, layer2_weights, [1, 2, 2, 1], padding='SAME')
    hidden = tf.nn.relu(conv + layer2_biases)
    shape = hidden.get_shape().as_list()
    reshape = tf.reshape(hidden, [shape[0], shape[1] * shape[2] * shape[3]])
    hidden = tf.nn.relu(tf.matmul(reshape, layer3_weights) + layer3_biases)
    return tf.matmul(hidden, layer4_weights) + layer4_biases
  
  # Training computation.
  logits = model(tf_train_dataset)
  loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels, logits=logits))
    
  # Optimizer.
  optimizer = tf.train.GradientDescentOptimizer(0.05).minimize(loss)
  
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits)
  valid_prediction = tf.nn.softmax(model(tf_valid_dataset))
  test_prediction = tf.nn.softmax(model(tf_test_dataset))


In [10]:
num_steps = 1001

with tf.Session(graph=graph) as session:
  tf.global_variables_initializer().run()
  print('Initialized')
  for step in range(num_steps):
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    batch_data = train_dataset[offset:(offset + batch_size), :, :, :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 50 == 0):
      print('Minibatch loss at step %d: %f' % (step, l))
      print('Minibatch accuracy: %.1f%%' % accuracy(predictions, batch_labels))
      print('Validation accuracy: %.1f%%' % accuracy(
        valid_prediction.eval(), valid_labels))
  print('Test accuracy: %.1f%%' % accuracy(test_prediction.eval(), test_labels))

  W1 = layer1_weights.eval(session)
  B1 = layer1_biases.eval(session)
  W2 = layer2_weights.eval(session)
  B2 = layer2_biases.eval(session)
  W3 = layer3_weights.eval(session)
  B3 = layer3_biases.eval(session)
  W4 = layer4_weights.eval(session)
  B4 = layer4_biases.eval(session)
    
g = tf.Graph()
with g.as_default():

  input = tf.placeholder(tf.float32, shape=(1, image_size, image_size, num_channels), name="input")

  W1 = tf.constant(W1, name="W1")
  B1 = tf.constant(B1, name="B1")
  W2 = tf.constant(W2, name="W2")
  B2 = tf.constant(B2, name="B2")
  W3 = tf.constant(W3, name="W3")
  B3 = tf.constant(B3, name="B3")
  W4 = tf.constant(W4, name="W4")
  B4 = tf.constant(B4, name="B4")

  # Model.
  def model(data):
    conv = tf.nn.conv2d(data, W1, [1, 2, 2, 1], padding='SAME')
    hidden = tf.nn.relu(conv + B1)
    conv = tf.nn.conv2d(hidden, W2, [1, 2, 2, 1], padding='SAME')
    hidden = tf.nn.relu(conv + B2)
    shape = hidden.get_shape().as_list()
    reshape = tf.reshape(hidden, [shape[0], shape[1] * shape[2] * shape[3]])
    hidden = tf.nn.relu(tf.matmul(reshape, W3) + B3)
    return tf.matmul(hidden, W4) + B4

  output = tf.nn.softmax(model(input), name="output")
    
  sess = tf.Session()
  init = tf.initialize_all_variables()
  sess.run(init)

  graph_def = g.as_graph_def()
  tf.train.write_graph(graph_def, '/tmp', '4_convolutions_model_graph.pb', as_text=False)


Initialized
Minibatch loss at step 0: 3.442225
Minibatch accuracy: 6.2%
Validation accuracy: 10.0%
Minibatch loss at step 50: 1.764352
Minibatch accuracy: 31.2%
Validation accuracy: 48.2%
Minibatch loss at step 100: 1.460109
Minibatch accuracy: 37.5%
Validation accuracy: 65.1%
Minibatch loss at step 150: 0.701383
Minibatch accuracy: 81.2%
Validation accuracy: 73.2%
Minibatch loss at step 200: 0.499818
Minibatch accuracy: 87.5%
Validation accuracy: 76.8%
Minibatch loss at step 250: 0.846414
Minibatch accuracy: 75.0%
Validation accuracy: 77.8%
Minibatch loss at step 300: 0.567223
Minibatch accuracy: 81.2%
Validation accuracy: 79.2%
Minibatch loss at step 350: 0.379122
Minibatch accuracy: 81.2%
Validation accuracy: 79.5%
Minibatch loss at step 400: 0.396388
Minibatch accuracy: 87.5%
Validation accuracy: 77.8%
Minibatch loss at step 450: 1.065120
Minibatch accuracy: 68.8%
Validation accuracy: 81.3%
Minibatch loss at step 500: 0.550759
Minibatch accuracy: 87.5%
Validation accuracy: 80.6%
Mi

---
Problem 1
---------

The convolutional model above uses convolutions with stride 2 to reduce the dimensionality. Replace the strides by a max pooling operation (`nn.max_pool()`) of stride 2 and kernel size 2.

---

In [20]:
batch_size = 512
patch_size = 5
depth = 32
depth2 = 64
num_hidden = 256
num_hidden2 = 128
pool_size = 3
pool_stride = 2
padding = 'SAME'

image_size_after = image_size // 4

graph = tf.Graph()

with graph.as_default():
  # Input data.
  tf_train_dataset = tf.placeholder(
    tf.float32, shape=(batch_size, image_size, image_size, num_channels))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  # Variables.
  layer1_weights = tf.Variable(tf.truncated_normal(
      [patch_size, patch_size, num_channels, depth], stddev=0.2))
  layer1_biases = tf.Variable(tf.constant(0.1, shape=[depth]))
  layer2_weights = tf.Variable(tf.truncated_normal(
      [patch_size, patch_size, depth, depth2], stddev=0.2))
  layer2_biases = tf.Variable(tf.constant(0.1, shape=[depth2]))

  layer3_weights = tf.Variable(tf.truncated_normal(
      [image_size_after * image_size_after * depth2, num_hidden], stddev=0.01))
  layer3_biases = tf.Variable(tf.constant(0.5, shape=[num_hidden]))
  layer4_weights = tf.Variable(tf.truncated_normal(
      [num_hidden, num_hidden2], stddev=0.01))
  layer4_biases = tf.Variable(tf.constant(0.5, shape=[num_hidden2]))
  layer5_weights = tf.Variable(tf.truncated_normal(
      [num_hidden2, num_labels], stddev=0.2))
  layer5_biases = tf.Variable(tf.constant(0.0, shape=[num_labels]))
 
  # Model.
  def model(data, keep_prob=1):
    conv = tf.nn.conv2d(data, layer1_weights, [1, 1, 1, 1], padding=padding)
    hidden = tf.nn.relu(conv + layer1_biases)
    pool = tf.nn.max_pool(hidden, [1, pool_size, pool_size, 1],[1, pool_stride, pool_stride, 1], padding=padding)
    
    conv = tf.nn.conv2d(pool, layer2_weights, [1, 1, 1, 1], padding=padding)
    hidden = tf.nn.relu(conv + layer2_biases)
    pool = tf.nn.max_pool(hidden,[1, pool_size, pool_size, 1],[1, pool_stride, pool_stride, 1], padding=padding)

    shape = pool.get_shape().as_list()
    reshape = tf.reshape(pool, [shape[0], shape[1] * shape[2] * shape[3]])
    
    hidden = tf.nn.dropout(tf.nn.relu(tf.matmul(reshape, layer3_weights) + layer3_biases), keep_prob)

    hidden = tf.nn.dropout(tf.nn.relu(tf.matmul(hidden, layer4_weights) + layer4_biases), keep_prob)
    
    return tf.matmul(hidden, layer5_weights) + layer5_biases

  # Training computation.
  logits = model(tf_train_dataset, keep_prob=0.8)
  loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels, logits=logits))

  # Optimizer.
  optimizer = tf.train.AdamOptimizer(0.001).minimize(loss)
  
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits)
  valid_prediction = tf.nn.softmax(model(tf_valid_dataset))
  test_prediction = tf.nn.softmax(model(tf_test_dataset))

In [21]:
num_steps = 5001

with tf.Session(graph=graph) as session:
  tf.global_variables_initializer().run()
  print('Initialized')
  for step in range(num_steps):
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    batch_data = train_dataset[offset:(offset + batch_size), :, :, :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 50 == 0):
      print('Minibatch loss at step %d: %f' % (step, l))
      print('Minibatch accuracy: %.1f%%' % accuracy(predictions, batch_labels))
      print('Validation accuracy: %.1f%%' % accuracy(
        valid_prediction.eval(), valid_labels))
  print('Test accuracy: %.1f%%' % accuracy(test_prediction.eval(), test_labels))


  W1 = layer1_weights.eval(session)
  B1 = layer1_biases.eval(session)
  W2 = layer2_weights.eval(session)
  B2 = layer2_biases.eval(session)
  W3 = layer3_weights.eval(session)
  B3 = layer3_biases.eval(session)
  W4 = layer4_weights.eval(session)
  B4 = layer4_biases.eval(session)
  W5 = layer5_weights.eval(session)
  B5 = layer5_biases.eval(session)
    
g = tf.Graph()
with g.as_default():

  input = tf.placeholder(tf.float32, shape=(1, image_size, image_size, num_channels), name="input")

  W1 = tf.constant(W1, name="W1")
  B1 = tf.constant(B1, name="B1")
  W2 = tf.constant(W2, name="W2")
  B2 = tf.constant(B2, name="B2")
  W3 = tf.constant(W3, name="W3")
  B3 = tf.constant(B3, name="B3")
  W4 = tf.constant(W4, name="W4")
  B4 = tf.constant(B4, name="B4")
  W5 = tf.constant(W5, name="W5")
  B5 = tf.constant(B5, name="B5")

  # Model.
  def model(data, keep_prob=1):
    conv = tf.nn.conv2d(data, W1, [1, 1, 1, 1], padding=padding)
    hidden = tf.nn.relu(conv + B1)
    pool = tf.nn.max_pool(hidden, [1, pool_size, pool_size, 1],[1, pool_stride, pool_stride, 1], padding=padding)
    
    conv = tf.nn.conv2d(pool, W2, [1, 1, 1, 1], padding=padding)
    hidden = tf.nn.relu(conv + B2)
    pool = tf.nn.max_pool(hidden,[1, pool_size, pool_size, 1],[1, pool_stride, pool_stride, 1], padding=padding)

    shape = pool.get_shape().as_list()
    reshape = tf.reshape(pool, [shape[0], shape[1] * shape[2] * shape[3]])
    
    hidden = tf.nn.dropout(tf.nn.relu(tf.matmul(reshape, W3) + B3), keep_prob)

    hidden = tf.nn.dropout(tf.nn.relu(tf.matmul(hidden, W4) + B4), keep_prob)
    
    return tf.matmul(hidden, W5) + B5

  output = tf.nn.softmax(model(input), name="output")
    
  sess = tf.Session()
  init = tf.initialize_all_variables()
  sess.run(init)

  graph_def = g.as_graph_def()
  tf.train.write_graph(graph_def, '/tmp', '4_convolutions_model_graph.pb', as_text=False)


Initialized
Minibatch loss at step 0: 2.955558
Minibatch accuracy: 10.5%
Validation accuracy: 10.0%
Minibatch loss at step 50: 0.791013
Minibatch accuracy: 78.1%
Validation accuracy: 82.5%
Minibatch loss at step 100: 0.503431
Minibatch accuracy: 84.0%
Validation accuracy: 85.8%
Minibatch loss at step 150: 0.539159
Minibatch accuracy: 83.4%
Validation accuracy: 87.2%
Minibatch loss at step 200: 0.485836
Minibatch accuracy: 84.4%
Validation accuracy: 88.3%
Minibatch loss at step 250: 0.371365
Minibatch accuracy: 89.6%
Validation accuracy: 88.8%
Minibatch loss at step 300: 0.409011
Minibatch accuracy: 88.3%
Validation accuracy: 88.9%
Minibatch loss at step 350: 0.397148
Minibatch accuracy: 87.9%
Validation accuracy: 89.3%
Minibatch loss at step 400: 0.447563
Minibatch accuracy: 87.3%
Validation accuracy: 90.0%
Minibatch loss at step 450: 0.415157
Minibatch accuracy: 89.3%
Validation accuracy: 90.0%
Minibatch loss at step 500: 0.385283
Minibatch accuracy: 87.7%
Validation accuracy: 90.2%
M

Validation accuracy: 92.7%
Minibatch loss at step 4550: 0.193297
Minibatch accuracy: 93.9%
Validation accuracy: 92.5%
Minibatch loss at step 4600: 0.163308
Minibatch accuracy: 94.5%
Validation accuracy: 92.3%
Minibatch loss at step 4650: 0.273442
Minibatch accuracy: 92.2%
Validation accuracy: 92.5%
Minibatch loss at step 4700: 0.188492
Minibatch accuracy: 93.0%
Validation accuracy: 92.6%
Minibatch loss at step 4750: 0.202046
Minibatch accuracy: 93.9%
Validation accuracy: 92.5%
Minibatch loss at step 4800: 0.217563
Minibatch accuracy: 93.9%
Validation accuracy: 92.6%
Minibatch loss at step 4850: 0.161031
Minibatch accuracy: 95.1%
Validation accuracy: 92.6%
Minibatch loss at step 4900: 0.221042
Minibatch accuracy: 92.4%
Validation accuracy: 92.5%
Minibatch loss at step 4950: 0.168764
Minibatch accuracy: 94.7%
Validation accuracy: 92.7%
Minibatch loss at step 5000: 0.166854
Minibatch accuracy: 93.6%
Validation accuracy: 92.8%
Test accuracy: 96.8%


---
Problem 2
---------

Try to get the best performance you can using a convolutional net. Look for example at the classic [LeNet5](http://yann.lecun.com/exdb/lenet/) architecture, adding Dropout, and/or adding learning rate decay.

---